# <center>A-T-L-A-S ATLAS!</center>

### Setup:

Install required libraries -

In [13]:
%pip install numpy pandas networkx matplotlib pyvis plotly python-louvain igraph leidenalg

Note: you may need to restart the kernel to use updated packages.


Libraries -

In [14]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import plotly.graph_objects as go

### Graph Creation:
create_atlas_graph() - Creates the graph based on rule "there exists an edge from A to B if you can say B after your opponent says A". A node ending with a letter x will have an outgoing edge to all nodes starting with a letter x.

In [15]:
def create_atlas_graph(data):
    """
    Creates a directed graph for the input.
    
    Parameters:
        data (list): A list of place names (countries/cities).
    
    Returns:
        G (networkx.DiGraph): A directed graph.
    """
    G = nx.DiGraph()
    
    for place in data:
        G.add_node(place)
        last_letter = place[-1].lower()  # last letter of the name, case-insensitive comparison
        for candidate in data:
            if candidate[0].lower() == last_letter:
                G.add_edge(place, candidate)
    
    return G

Read data from datasets and create graphs for each dataset.

In [16]:
with open("/Users/mago/Desktop/Atlas/data/countries.txt", "r") as file:
    countries = [line.strip() for line in file]

with open("/Users/mago/Desktop/Atlas/data/cities.txt", "r") as file:
    cities = [line.strip() for line in file]

combined = countries + cities

country_graph = create_atlas_graph(countries)
city_graph = create_atlas_graph(cities)
combined_graph = create_atlas_graph(combined)

### Graph Visualisation:

Interactive graph code, produces an interactive graph webpage -

In [17]:
def visualize_interactive_graph(G, title):
  """
  Visualizes a directed graph using NetworkX, Pyvis and Matplotlib.

  Parameters:
      G (networkx.DiGraph): A directed graph.
      title (str): Title for the graph visualization.
      save_path (str, optional): Path to save the visualization as an image file. Defaults to None.
      interactive (bool, optional): If True, creates an interactive graph using pyvis. Defaults to False.
  """
  net = Network(height="100%", width="100%", notebook=True, directed=True)
  net.from_nx(G)
  net.set_options("""
    var options = {
      "physics": {
        "enabled": true,
        "stabilization": {
          "enabled": true,
          "iterations": 1000
        },
        "solver": "forceAtlas2Based",
        "forceAtlas2Based": {
          "gravitationalConstant": -50,
          "springLength": 100,
          "springConstant": 0.08,
          "avoidOverlap": 1
        }
      }
    }
  """)
  net.show(f"{title.replace(' ', '_').lower()}.html")
  return

In [18]:
visualize_interactive_graph(country_graph, "Country Graph")
# visualize_interactive_graph(city_graph, "City Graph")
# visualize_interactive_graph(combined_graph, "Combined Graph")

country_graph.html


Static 3D graph plotted model -

In [19]:
def visualize_static_graph(G, title):
    """
    Visualizes a static 3D graph using Networkx, Plotly and Matplotlib.
    
    Parameters:
        G (networkx.Graph): The graph to visualize.
        title (str): Title of the graph.
    """
    # Generate a 3D layout for the graph
    pos = nx.spring_layout(G, dim=3, seed=42)  # 3D spring layout
    
    # Extract node positions
    x_nodes = [pos[node][0] for node in G.nodes()]
    y_nodes = [pos[node][1] for node in G.nodes()]
    z_nodes = [pos[node][2] for node in G.nodes()]
    
    # Create edge traces
    x_edges = []
    y_edges = []
    z_edges = []
    
    for edge in G.edges():
        x_edges += [pos[edge[0]][0], pos[edge[1]][0], None]
        y_edges += [pos[edge[0]][1], pos[edge[1]][1], None]
        z_edges += [pos[edge[0]][2], pos[edge[1]][2], None]
    
    # Create node trace
    node_trace = go.Scatter3d(
        x=x_nodes, y=y_nodes, z=z_nodes,
        mode='markers',
        marker=dict(
            size=5,
            color=np.arange(len(G.nodes())),  # Color by node index
            colorscale='Viridis',  # Color scheme
            opacity=0.8
        ),
        text=list(G.nodes()),  # Node labels
        hoverinfo='text'
    )
    
    # Create edge trace
    edge_trace = go.Scatter3d(
        x=x_edges, y=y_edges, z=z_edges,
        mode='lines',
        line=dict(color='gray', width=1),
        hoverinfo='none'
    )
    
    # Create the figure
    fig = go.Figure(
        data=[edge_trace, node_trace],
        layout=go.Layout(
            title=title,
            showlegend=False,
            scene=dict(
                xaxis=dict(showbackground=False),
                yaxis=dict(showbackground=False),
                zaxis=dict(showbackground=False)
            )
        )
    )
    
    # Show the figure
    fig.show()

In [20]:
# visualize_static_graph(city_graph, "City Graph - Static 3D Visualization")
visualize_static_graph(country_graph, "Country Graph - Static 3D Visualization")

### Community Detection:

Girvan-Newman (Edge Betweeness Centrality) -

In [21]:
from networkx.algorithms.community import girvan_newman

def detect_communities_girvan_newman(G):
    """
    Applies the Girvan-Newman algorithm to detect communities in a graph.

    Parameters:
        G (networkx.Graph): The graph to analyze.

    Returns:
        communities (list): A list of sets, where each set represents a community.
    """
    comp = girvan_newman(G)
    first_level_communities = next(comp)
    communities = [list(c) for c in first_level_communities]

    print("Detected ", len(communities), " communities using Girvan-Newman at depth.")
    return communities

In [22]:
girvan_newman_countries = detect_communities_girvan_newman(country_graph)
print(girvan_newman_countries)

# girvan_newman_cities = detect_communities_girvan_newman(city_graph)
# print(girvan_newman_cities)

# girvan_newman_combined = detect_communities_girvan_newman(combined_graph)
# print(girvan_newman_combined)

Detected  2  communities using Girvan-Newman at depth.
[['South Korea', 'Uzbekistan', 'Malaysia', 'The Bahamas', 'Thailand', 'North Korea', 'Jamaica', 'Guinea-Bissau', 'Samoa', 'Nepal', 'Algeria', 'Norway', 'Yemen', 'Tunisia', 'Democratic Republic of the Congo', 'Mali', 'Liberia', 'Albania', 'Mauritania', 'Slovenia', 'Bhutan', 'Bulgaria', 'Cuba', 'Switzerland', 'Tuvalu', 'Hungary', 'Bangladesh', 'Ecuador', 'Lesotho', 'Netherlands', 'Kazakhstan', 'Finland', 'Poland', 'Belize', 'India', 'Vatican City', 'Saudi Arabia', 'El Salvador', 'Belgium', 'France', 'Egypt', 'San Marino', 'Mauritius', 'Turkmenistan', 'Mexico', 'Canada', 'Syria', 'Namibia', 'South Sudan', 'Andorra', 'Nigeria', 'Pakistan', 'China', 'Sierra Leone', 'Jordan', 'Ethiopia', 'Morocco', 'Djibouti', 'Spain', 'Cameroon', 'Liechtenstein', 'Brazil', 'Sweden', 'Gabon', 'Israel', 'Zimbabwe', 'Madagascar', 'Latvia', 'Senegal', 'Brunei', 'United Arab Emirates', 'Papua New Guinea', 'Trinidad and Tobago', 'Tonga', 'Tanzania', 'Argentin

Girvan-Newman with number of communities constraint -

In [23]:
def detect_communities_girvan_newman_with_split(G, num_splits=5):
    """
    Applies the Girvan-Newman algorithm and returns multiple levels of community detection.

    Parameters:
        G (networkx.Graph): The graph to analyze.
        num_splits (int): The number of splits to perform.

    Returns:
        best_communities (list): List of detected communities at the chosen split.
    """
    comp = girvan_newman(G)

    # Take the nth split (instead of just the first one)
    for i in range(num_splits):
        communities = next(comp)

    best_communities = [list(c) for c in communities]  # Convert sets to lists
    print("Detected ", len(best_communities), " communities using Girvan-Newman with ", num_splits, " splits.")

    return best_communities

In [24]:
country_split = 10
city_split = 100
combined_split = 150

girvan_newman_countries = detect_communities_girvan_newman_with_split(country_graph, num_splits=country_split)
print(girvan_newman_countries)

for i in range (country_split):
    graph = create_atlas_graph(girvan_newman_countries[i])
    visualize_static_graph(graph, "graph %i"%(i+1))

# girvan_newman_cities = detect_communities_girvan_newman_with_split(city_graph, num_splits=city_split)
# print(girvan_newman_cities)

# for i in range (city_split):
#     graph = create_atlas_graph(girvan_newman_cities[i])
#     visualize_static_graph(graph, "graph %i"%(i+1))

# girvan_newman_combined = detect_communities_girvan_newman_with_split(combined_graph, num_splits=combined_split)
# print(girvan_newman_combined)

# for i in range (combined_split):
#     graph = create_atlas_graph(girvan_newman_combined[i])
#     visualize_static_graph(graph, "graph %i"%(i+1))

Detected  11  communities using Girvan-Newman with  10  splits.
[['South Korea', 'Uzbekistan', 'Malaysia', 'The Bahamas', 'Thailand', 'North Korea', 'Jamaica', 'Guinea-Bissau', 'Samoa', 'Nepal', 'Algeria', 'Norway', 'Yemen', 'Tunisia', 'Democratic Republic of the Congo', 'Mali', 'Liberia', 'Albania', 'Mauritania', 'Slovenia', 'Bhutan', 'Bulgaria', 'Cuba', 'Switzerland', 'Tuvalu', 'Ecuador', 'Lesotho', 'Netherlands', 'Kazakhstan', 'Belize', 'India', 'Saudi Arabia', 'El Salvador', 'Belgium', 'France', 'Egypt', 'San Marino', 'Mauritius', 'Turkmenistan', 'Mexico', 'Canada', 'Syria', 'Namibia', 'South Sudan', 'Andorra', 'Nigeria', 'Pakistan', 'China', 'Sierra Leone', 'Jordan', 'Ethiopia', 'Morocco', 'Djibouti', 'Spain', 'Cameroon', 'Liechtenstein', 'Brazil', 'Sweden', 'Gabon', 'Israel', 'Zimbabwe', 'Madagascar', 'Latvia', 'Senegal', 'Brunei', 'United Arab Emirates', 'Papua New Guinea', 'Tonga', 'Tanzania', 'Argentina', 'Greece', 'Maldives', 'Cambodia', 'Kuwait', 'Honduras', 'Saint Vincent a

Leiden -

In [25]:
import leidenalg
import igraph as ig

def detect_communities_leiden(G):
    """
    Applies the Leiden method for community detection.

    Parameters:
        G (networkx.Graph): The graph to analyze.

    Returns:
        communities (list): A list of detected communities.
    """
    # Convert NetworkX graph to iGraph
    nx_to_igraph = ig.Graph.TupleList(G.edges(), directed=True)

    # Apply Leiden algorithm
    partition = leidenalg.find_partition(nx_to_igraph, leidenalg.ModularityVertexPartition)

    num_communities = len(partition)
    print("Detected ", num_communities, " communities using Leiden Algorithm.")

    # Convert to list format
    communities = [list(map(str, comm)) for comm in partition]

    return communities

In [26]:
leiden_countries = detect_communities_leiden(country_graph)

for i in range (len(leiden_countries)):
    graph = create_atlas_graph(leiden_countries[i])
    visualize_static_graph(graph, "graph %i"%(i+1))

leiden_cities = detect_communities_leiden(city_graph)

for i in range (len(leiden_cities)):
    graph = create_atlas_graph(leiden_cities[i])
    visualize_static_graph(graph, "graph %i"%(i+1))

Detected  6  communities using Leiden Algorithm.


Detected  7  communities using Leiden Algorithm.
